# Solving-Hamilton-Jacobi-Bellman Equations
#### Frederik Kelbel, Imperial College London

## Dependencies

In [1]:
import torch
import plotly.graph_objects as go
import numpy as np
from operators import div, Δ, D, mdotb, bdotm, mdotm, bdotb
from DGM import DGMSolver, DGMPIASolver
from pdes import HBJ
from scipy.integrate import quad
from plotly.subplots import make_subplots
from configs import CONFIG_HBJS as MODEL_CONFIG
from itertools import product

## Plotting

In [2]:
def plot_losses(losses, avg_over=10):
    avgs_1 = np.convolve(losses[:, 0], np.ones(avg_over), 'valid') / avg_over
    avgs_2 = np.convolve(losses[:, 1], np.ones(avg_over), 'valid') / avg_over
    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Scatter(x=np.arange(len(avgs_1)), y=avgs_1, mode='lines', name="Value Loss"), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(len(avgs_2)), y=avgs_2, mode='lines', name="Control Loss"), row=1, col=1)
    fig.update_layout(
        title="Loss",
        xaxis_title="Iterations",
        yaxis_title="Loss",
        font=dict(
            family="Courier New, monospace",
            size=14
        )
    )
    fig.show()
    
def plot_value(solver, sol):
    fig = make_subplots(rows=1, cols=2, 
                   specs=[[{'type': 'surface'}, {'type': 'surface'}]])
    xs = np.linspace(0, 1, 100)
    ts = np.linspace(0.01, 1, 100)
    us_pred = np.array([[solver(x, t).item() for x in xs] for t in ts])
    us = np.array([[sol(x, t) for x in xs] for t in ts])
    fig.add_trace(go.Surface(z=us, showscale=False), row=1, col=1)
    fig.add_trace(go.Surface(z=us_pred), row=1, col=2)
    fig.update_layout(title='Solution | Approximation',
                  scene = dict(
                    xaxis_title="t",
                    yaxis_title="x",
                    zaxis_title="J(x, t"),
                  scene2 = dict(
                    xaxis_title="t",
                    yaxis_title="x",
                    zaxis_title="J(x, t)"),
                  margin=dict(l=50, r=50, b=50, t=50))
    fig.show()

## Problem Formulation

Objective: Find the control process $u = (u_t)_{t \geq 0}$ in admissable set $\mathcal{A}$ for an Itô Process $X^u = (X_t^u)_{t \geq 0}$ satisfying:


$$
d X_t^u = \mu(t, X_t^u, u_t) dt + \sigma(t, X_t^u, u_t) d W_t, \quad X_0^u = 0.
$$

We will consider the HBJ-Equations in their primal form.

The agents performance is assessed via:
$$
J^u(t, x) = \mathbb{E}\Big[ \int_t^T F(s, X_s^u, u_s) ds + G(X_T^u) \;\Big|\; X_t^u = x \Big]
$$

Denote $J(t, x) = \sup_{u \in \mathcal{A}} J^u(t, x)$, then this value function satisfies the following HJB-equations:

$$
\begin{cases}
\partial_t J(t, x) + \sup_{u \in \mathcal{A}} \{\mathscr{L}^u_t J(t, x) + F(t, x, u)\} = 0 \\
J(T, x) = G(x)
\end{cases}
$$

### The Merton Problem (Wealth Allocation Problem)

The goal is to find the optimal wealth allocation strategy over time such that the wealth itself is maximized.

Consider a market with a risky and risk-free asset. Suppose the value of the risk-free asset at time $t$ is given by $\frac{d B_t}{B_t} = r dt$ or $B_t = B_0 e^{rt}, t\geq 0$. Additionally, we have that the risky asset evolves accordingly to $\frac{d S_t}{S_t} = \mu dt + \sigma dW_t$, where $\{W_t\}_{t\geq 0}$ is a standard one-dimensional Brownian motion.

The wealth can then be described by
$$
X_t= x + \int_0^t v_s \frac{dS_s}{S_s} + \int_0^t(X_s-v_s)\frac{dB_s}{B_s},
$$
where $v_t = u_t X_t$ describes the amount of the wealth to be have invested into the risky asset at time $t$. $u_t$ is the fraction of wealth invested in the risky asset.

Let $(\Omega, \mathcal{F}, \{\mathcal{F}_t\}_{t\in [0, T]}, \mathbb{P})$ be a filtered probability space. The evolution of an investor's wealth is described via
$$
\begin{cases}
dX_s = ((\mu -r)u_s + r)X_s ds + \sigma u_s X_s dW_s, \; s \in [0, T] \\
X_0 = x > 0
\end{cases},
$$

with $\mu>r$ and $\sigma>0$ referring to drift and volatility, respectively. Let $r>0$ denote the discount rate, i.e. the depreciation constant. The intend is to maximise the objective
$$
J^u(t, X_t) = \mathbb{E}[ x^\gamma ], \; \gamma \in(0, 1).
$$

The respective HBJ-equation becomes:

$$
\begin{cases}
\partial_t J(t, x) + \sup_{u} \Big\{ ((\mu-r)u + r)x \partial_x J(t, x) + \frac{1}{2} \sigma^2 u^2 x^2\partial_{xx} J(t, x) \Big\} = 0 \text{ on $[0, T] \times (0, \infty)$}
\\
J(T, x) = x^\gamma \text{ $\forall x > 0$}
\end{cases}
$$

#### Analytical Solution (Oksendal):

Assume $J(t, x) = w(t) v(x)$, with $J(T, x) = w(T) v(x) = x^\gamma$. We guess $J(t, x) = w(t) x^\gamma$, where $w(T)=1$. The problem becomes 
$$
\begin{cases}
w'(t) + \gamma  \sup_{u} \Big\{ (\mu-r) u + r + \frac{1}{2} \sigma^2 u^2 (\gamma-1) \Big\}w(t) = 0
\\
w(T) = x^\gamma
\end{cases}.
$$

Then, $u$ is maximised for $u^* = \frac{\mu-r}{\sigma^2 (1-\gamma)}$ and the equation becomes
$$
\begin{cases}
w'(t) + \frac{\gamma (\mu-r)^2}{\sigma^2(2-2\gamma)}w(t) = 0
\\
w(T) = x^\gamma
\end{cases}.
$$
Thus, we have  $w'(t) = \frac{\gamma (\mu-r)^2}{\sigma^2(2\gamma - 2)}w(t)$. It follows that $w(t) = \exp\big(\frac{\gamma (\mu-r)^2}{\sigma^2(2\gamma - 2)}t\big)$. The final solution is:
$$
J(t, x) = w(t)v(x) = \exp\big(\frac{\gamma (\mu-r)^2}{\sigma^2(2\gamma - 2)}t\big) x^\gamma.
$$

We need to verify this using the HBJ-verification theorem.

In [ ]:
class RISKY_ASSET(HBJ):
    def __init__(self):
        super().__init__()
        self.μ = 0.04
        self.σ = 0.4
        self.r = 0.03
        self.γ = 0.5
        
        def terminal_func(var):
            var[-1] = 0*var[-1] + 1
            return var
        
        self.var_dim_J = 2 # (x, t)
        self.control_vars = [1] # (t)
        self.cost_function = lambda u, var: 0
        self.differential_operator = lambda J, u, var: ((self.μ-self.r)*u+self.r)*var[0]*div(J, var[0]) + 0.5*self.σ**2*u**2*var[0]**2*Δ(J, var[0])
        self.domain_func = [(lambda var: var, 128)]
        self.boundary_cond_J = [lambda J, var: J - var[0]**self.γ]
        self.boundary_func_J = [(lambda var: terminal_func(var), 128)]
        self.boundary_cond_u = [lambda u, var: torch.clamp(u, min=1) - torch.clamp(u, max=-1) - 2]
        self.boundary_func_u = [(lambda var: var, 64)]

In [ ]:
eq = RISKY_ASSET()
model = MODEL_CONFIG
solver = DGMPIASolver(model, eq)
loss = np.array(list(solver.train(800)))
plot_losses(loss)

In [ ]:
u_sol = lambda t : (eq.μ-eq.r)/(eq.σ**2*(1-eq.γ))
J_sol = lambda x, t: x**eq.γ * np.exp((eq.γ*(eq.μ-eq.r)**2)/(eq.σ**2*(2*eq.γ-2))*t)

In [ ]:
fig = make_subplots(rows=1, cols=1)
eval_points = np.linspace(0, 1, 100)
fig.add_trace(go.Scatter(x=eval_points, y=[solver.u(p)[0] for p in eval_points], mode='lines', name="Optimal Control"), row=1, col=1)
fig.add_trace(go.Scatter(x=eval_points, y=[u_sol(p) for p in eval_points], mode='lines', name="Optimal Control Solution"), row=1, col=1)
fig.update_layout(
    title="Solutions",
    xaxis_title="Time",
    yaxis_title="Control Signal",
    yaxis_range=[0, 1],
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()

In [ ]:
plot_value(solver.J, J_sol)

### Linear-quadratic control problem 1-dimensional (Riccati Equation) 

Let $(\Omega, \mathcal{F}, \{\mathcal{F}_t\}_{t\in [0, T]}, \mathbb{P})$. We consider
$$
\begin{cases}
dX_s = [H_sX_s + M_s u_s] ds + \sigma_s dW_s, \; s \in [0, T] \\
X_0 = x > 0
\end{cases},
$$

We aim to maximise
$$
J^u(t, x) := \mathbb{E}^{t, x} \Big[ \int_t^T X_s^T C_s X_s + u_s^T D_s u_s ds + X_T^T R X_T\Big],
$$
with $C(t) = C \leq 0, R \leq 0$, and $D=D(t) < -\delta < 0$ given and deterministic ($\delta > 0$ some constant).

We write down the problem in its primal form as
$$
\begin{cases}
\partial_t J(t, x) + \sup_{u} \Big\{ \frac{1}{2} \sigma^2 \partial_{xx} J(t, x) + [H x + M u] \partial_x J(t, x) + C x^2 + D u^2 \Big\} = 0 \text{ on $[0, T] \times (0, \infty)$}
\\
J(T, x) = Rx^2 \text{ $\forall x \in \mathbb{R}$}
\end{cases}
$$

#### Analytical Solution (Oksendal):

As $J(T, x) = Rx^2$, we assume the form $J(T, x) = S(t) x^2 + b(t)$ for some differentibale $S$ and $b$. The problem can be reformulated as
$$
\begin{cases}
S'(t)x^2 + b'(t) + \sigma^2 S(t) + 2HS(t)x^2 + C x^2 + \sup_{u} \Big\{ 2MS(t) u x + D u^2 \Big\} = 0 \text{ on $[0, T] \times (0, \infty)$}
\\
S(T) = R, b(t)=0 \text{ $\forall x \in \mathbb{R}$}
\end{cases}
$$

We see that for fixed $t$ and $x$, we have $u^*=u^*(t, x)=-D^{-1} MS(t) x$. As a consequence, we can transform the problem to be gives as
$$
\begin{cases}
[S'(t) + 2HS(t) + C - D^{-1} M^2S^2(t)]x^2 + b'(t) + \sigma^2 S(t) = 0 \text{ on $[0, T] \times (0, \infty)$}
\\
S(T) = R, b(t)=0 \text{ $\forall x \in \mathbb{R}$}
\end{cases}
$$

The problem is satisfied if
$$
S'(t)= D^{-1} M^2S^2(t) - C - 2HS(t), \; S(T)=R \; \text{(Riccati Equation)}
$$
$$
b'(t)= -\sigma^2 S(t), \; b(T)=0
$$

It follows that $b(t)=  \sigma^2 \int_t^T S(r) dr$, while the Riccati Equation has a unique solution.

In [ ]:
class LQR(HBJ):
    def __init__(self):
        super().__init__()
        self.H = 1.0
        self.M = 1.2
        self.C = -1.0
        self.R = -1.5
        self.D = -0.05
        self.σ = 0.0
        
        self.var_dim_J = 2 # (x, t)
        self.control_vars = [0, 1]
        self.cost_function = lambda u, var: self.C*var[0]**2 + self.D*u**2
        self.differential_operator = lambda J, u, var: (self.H*var[0] + self.M*u)*div(J, var[0]) + (1/2)*self.σ**2*Δ(J, var[0])
        self.domain_func = [(lambda var: [var[0]*2-1, var[1]], 128)]
        self.boundary_cond_J = [lambda J, var: J - self.R*var[0]**2]
        self.boundary_func_J = [(lambda var: [var[0]*2-1, var[1]*0 + 1], 64)]

In [ ]:
LQR_MODEL_CONFIG = {
    "hidden_dim": 128,
    "learning_rate": 5e-3,
    "loss_weights": (1, 1),
    "lr_decay": 0.99,
    "sampling_method": "uniform",
    "network_type": "GRU",
    "optimiser": "Adam"
}
eq = LQR()
model = LQR_MODEL_CONFIG
solver = DGMPIASolver(model, eq)
loss = np.array(list(solver.train(800)))
plot_losses(loss)

#### Control function approximation:

In [ ]:
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}]])
xs = np.linspace(-3, 3, 100)
ts = np.linspace(0, 1, 100)
us_pred = np.array([[solver.u(x, t).item() for x in xs] for t in ts])
fig.add_trace(go.Surface(z=us_pred), row=1, col=1)
fig.update_layout(title='Approximation',
                  scene = dict(
                      xaxis_title="t",
                      yaxis_title="x",
                      zaxis_title="u(x, t"),
                  margin=dict(l=50, r=50, b=50, t=50))
fig.show()

#### Value function approximation:

In [ ]:
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}]])
xs = np.linspace(-3, 3, 100)
ts = np.linspace(0, 1, 100)
us_pred = np.array([[solver.J(x, t).item() for x in xs] for t in ts])
fig.add_trace(go.Surface(z=us_pred), row=1, col=1)
fig.update_layout(title='Solution | Approximation',
                  scene = dict(
                    xaxis_title="t",
                    yaxis_title="x",
                    zaxis_title="J(x, t"),
                  scene2 = dict(
                    xaxis_title="t",
                    yaxis_title="x",
                    zaxis_title="J(x, t)"),
                  margin=dict(l=50, r=50, b=50, t=50))
fig.show()

#### Simulating the Process:

Let $(\Omega, \mathcal{F}, \{\mathcal{F}_t\}_{t\in [0, T]}, \mathbb{P})$. We consider
$$
\begin{cases}
dX_s = [H_s X_s + M_s u_s] ds + \sigma_s dW_s, \; s \in [0, T] \\
X_0 = x > 0
\end{cases},
$$

In [ ]:
n = 50
c_xs = np.zeros(n)
c_xs[0] = 0.75
uc_xs = np.zeros(n)
uc_xs[0] = 0.75
dt = 1/n
ts = [t for t in np.linspace(0, 1, n)]
c_cum_cost = np.zeros(n)
uc_cum_cost = np.zeros(n)
for i in range(n-1):
    dW = np.sqrt(dt)*np.random.randn()
    c = solver.u(c_xs[i], i*dt).item()
    uc = np.random.rand()-0.5
    c_xs[i+1] = c_xs[i] + (eq.H*c_xs[i] + eq.M*c)*dt + eq.σ*dW
    uc_xs[i+1] = uc_xs[i] + (eq.H*uc_xs[i] + eq.M*uc)*dt + eq.σ*dW
    c_cum_cost[i+1] = c_cum_cost[i] + eq.C*c_xs[i]**2 + eq.D*c**2
    uc_cum_cost[i+1] = uc_cum_cost[i] + eq.C*uc_xs[i]**2 + eq.D*uc**2

c_cum_cost[-1] = c_cum_cost[-2] + eq.R*c_xs[-1]**2
uc_cum_cost[-1] = uc_cum_cost[-2] + eq.R*uc_xs[-1]**2
    
fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=ts, y=c_xs, mode='lines', name="Controlled", line=dict(color="#00e476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=uc_xs, mode='lines', name="Uncontrolled", line=dict(color="#FFe476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=c_cum_cost, mode='lines', showlegend=False, line=dict(color="#00e476")), row=1, col=2)
fig.add_trace(go.Scatter(x=ts, y=uc_cum_cost, mode='lines', showlegend=False, line=dict(color="#FFe476")), row=1, col=2)
fig.update_layout(
    title="Minimise amount of X | Minimise the costs (hold both close to zero)",
    xaxis_title="t",
    yaxis_title="X",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()

Example: Assume that the UK is subject to another disease outbreak. Each patient admission costs certain resources. We intend to reduce the amount of people in hospitals as quickly as possible to bound admission costs.

### Linear-quadratic control problem N-dimensional (Riccati Equation) 

We consider the same problem with $H_t \in \mathbb{R}^{n \times n}$,
$M_t \in \mathbb{R}^{n \times k}$, $\sigma_t \in \mathbb{R}^{n \times m}$ (for now a scalar), $C_t \in \mathbb{R}^{n \times n}$, $D_t \in \mathbb{R}^{k \times n}$, and $R \in \mathbb{R}^{n \times n}$. We also have, $u(t, X_t) \in\mathbb{R}^k$.

Let $(\Omega, \mathcal{F}, \{\mathcal{F}_t\}_{t\in [0, T]}, \mathbb{P})$. We consider
$$
\begin{cases}
dX_s = [H_sX_s + M_s u_s] ds + \sigma_s dW_s, \; s \in [0, T] \\
X_0 = x > 0
\end{cases},
$$

We aim to maximise
$$
J^u(t, x) := \mathbb{E}^{t, x} \Big[ \int_t^T X_s^T C_s X_s + u_s^T D_s u_s ds + X_T^T R X_T\Big],
$$
with $C(t) = C \leq 0, R \leq 0$, and $D=D(t) < -\delta < 0$ given and deterministic ($\delta > 0$ some constant).

We write down the problem in its primal form as
$$
\begin{cases}
\partial_t J(t, x) + \sup_{u} \Big\{ \frac{1}{2} \sigma^2 \sum_{1 \leq i, j \leq n} \Delta_{x_i x_j} J(t, x) + [H x + M u]^T \cdot \nabla_x J(t, x) + x^T C x + u^T D u \Big\} = 0 \text{ on $[0, T] \times (0, \infty)$}
\\
J(T, x) = x^T R x \text{ $\forall x \in \mathbb{R}$}
\end{cases}
$$

For now, we will consider the problem without the terminal condition.

In [61]:
class LQR_N(HBJ):
    def __init__(self):
        super().__init__()
        self.H = torch.tensor([[1.0, 0],
                              [0.0, 1.0]])
        self.M = torch.tensor([[1.2, 0],
                              [0.0, 1.2]])
        self.C = torch.tensor([[-1.2, 0],
                              [0.0, -1.0]])
        #self.R = torch.tensor([[-1.5, 0],
                              #[0.0, -1.5]])
        self.D = torch.tensor([[-0.05, 0],
                              [0.0, -0.05]])
        self.σ = 0.3
        
        self.var_dim_J = 3 # (x, y, t)
        self.sol_dim = 2
        self.control_vars = [0, 1, 2]
        self.cost_function = lambda u, var: (var[:2] |bdotb| (self.C |mdotb| var[:2])) + ((u |bdotm| self.D) |bdotb| u)
        self.differential_operator = lambda J, u, var: (((self.H |mdotb| var[:2]) + (self.M |mdotb| u)) |bdotb| D(J, var[:2])) + (1/2)*self.σ**2*sum([div(div(J, var[i]), var[j]) for i, j in product(range(2), range(2))])
        self.domain_func = [(lambda var: [(var[0]-0.5)*4, (var[1]-0.5)*4, var[2]], 128)]

In [62]:
LQR_N_MODEL_CONFIG = {
    "hidden_dim": 128,
    "learning_rate": 5e-3,
    "loss_weights": (1, 0),
    "lr_decay": 0.99,
    "sampling_method": "uniform",
    "network_type": "GRU",
    "optimiser": "Adam"
}
eq = LQR_N()
model = LQR_N_MODEL_CONFIG
solver = DGMPIASolver(model, eq)
loss = np.array(list(solver.train(500)))
plot_losses(loss)

100%|██████████| 500/500 [01:12<00:00,  6.92 it/s]


#### Simulating the Process:

Let $(\Omega, \mathcal{F}, \{\mathcal{F}_t\}_{t\in [0, T]}, \mathbb{P})$. We consider
$$
\begin{cases}
dX_s = [H_sX_s + M_s u_s] ds + \sigma_s dW_s, \; s \in [0, T] \\
X_0 = x > 0
\end{cases},
$$

In [63]:
n = 50
c_xs = np.zeros((2, n))
c_xs[:, 0] = -0.5
uc_xs = np.zeros((2, n))
uc_xs[:, 0] = -0.5
dt = 1/n
ts = [t for t in np.linspace(0, 1, n)]
c_cum_cost = np.zeros(n)
uc_cum_cost = np.zeros(n)
for i in range(n-1):
    dW = np.sqrt(dt)*np.random.randn(2, 1)
    c = -np.expand_dims(solver.u(c_xs[0, i], c_xs[1, i], i*dt), axis=1)
    uc = (np.random.rand(2, 1)-0.5)*2
    c_xs[:, None, i+1] = c_xs[:, None, i] + (eq.H.numpy() @ c_xs[:, None, i] + eq.M.numpy() @ c)*dt + eq.σ*dW
    uc_xs[:, None, i+1] = uc_xs[:, None, i] + (eq.H.numpy() @ uc_xs[:, None, i] + eq.M.numpy() @ uc)*dt + eq.σ*dW
    c_cum_cost[i+1] = c_cum_cost[i] + c_xs[:, None, i].T @ eq.C.numpy() @ c_xs[:, None, i] + c.T @ eq.D.numpy() @ c
    uc_cum_cost[i+1] = uc_cum_cost[i] + uc_xs[:, None, i].T @ eq.C.numpy() @ uc_xs[:, None, i] + uc.T @ eq.D.numpy() @ uc

c_cum_cost[-1] = c_cum_cost[-2] #+ c_xs[:, None, -1].T @ eq.R.numpy() @ c_xs[:, None, -1]
uc_cum_cost[-1] = uc_cum_cost[-2] #+ uc_xs[:, None, -1].T @ eq.R.numpy() @ uc_xs[:, None, -1]
fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=ts, y=c_xs[0], mode='lines', name="Controlled", line=dict(color="#00e476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=c_xs[1], mode='lines', showlegend=False, line=dict(color="#00e476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=uc_xs[0], mode='lines', name="Uncontrolled", line=dict(color="#FFe476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=uc_xs[1], mode='lines', showlegend=False, line=dict(color="#FFe476")), row=1, col=1)
fig.add_trace(go.Scatter(x=ts, y=c_cum_cost, mode='lines', showlegend=False, line=dict(color="#00e476")), row=1, col=2)
fig.add_trace(go.Scatter(x=ts, y=uc_cum_cost, mode='lines', showlegend=False, line=dict(color="#FFe476")), row=1, col=2)
fig.update_layout(
    title="Minimise amount of X | Minimise the costs (hold both close to zero)",
    xaxis_title="t",
    yaxis_title="X",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()